In [1]:
import boto3
s3_resource = boto3.resource('s3')

## Creating a Bucket

To create one programmatically, you must first choose a name for your bucket. Remember that this name must be unique throughout the whole AWS platform, as bucket names are DNS compliant.

You can generate your own function that does that for you. In this implementation, you’ll see how using the uuid module will help you achieve that. A UUID4’s string representation is 36 characters long (including hyphens), and you can add a prefix to specify what each bucket is for.

In [5]:
import uuid
def create_bucket_name(bucket_prefix):
    # The generated bucket name must be between 3 and 63 chars long
    return ''.join([bucket_prefix, str(uuid.uuid4())])

Boto3 will create the session from your credentials. 

You just need to take the region and pass it to create_bucket() as its LocationConstraint configuration. 

The nice part is that this code works no matter where you want to deploy it: locally/EC2/Lambda. Moreover, you don’t need to hardcode your region.

In [6]:
def create_bucket(bucket_prefix, s3_connection):
    session = boto3.session.Session()
    current_region = session.region_name
    bucket_name = create_bucket_name(bucket_prefix)
    bucket_response = s3_connection.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': current_region})
    
    print(bucket_name, current_region)
    return bucket_name, bucket_response

You’ll now create two buckets. 

First create one using the client, which gives you back the bucket_response as a dictionary:

In [10]:
first_bucket_name, first_response = create_bucket(
                                    bucket_prefix='firstpythonbucket',
                                    s3_connection=s3_resource.meta.client)

ParamValidationError: Parameter validation failed:
Invalid type for parameter CreateBucketConfiguration.LocationConstraint, value: None, type: <class 'NoneType'>, valid types: <class 'str'>

In [9]:
first_response

NameError: name 'first_response' is not defined

Then create a second bucket using the resource, which gives you back a Bucket instance as the bucket_response:

In [6]:
second_bucket_name, second_response = create_bucket(
                                                    bucket_prefix='secondpythonbucket',
                                                    s3_connection=s3_resource)

secondpythonbucket6ce9cccf-c429-471c-99a1-f36e849ee381 us-east-2


In [7]:
second_response

s3.Bucket(name='secondpythonbucket6ce9cccf-c429-471c-99a1-f36e849ee381')

## Creating Bucket and Object Instances

The next step after creating your file is to see how to integrate it into your S3 workflow.
This is where the resource’s classes play an important role, as these abstractions make it easy to work with S3.

By using the resource, you have access to the high-level classes (Bucket and Object).

In [9]:
second_bucket = s3_resource.Bucket(name=second_bucket_name)
file_object = s3_resource.Object(bucket_name=second_bucket_name,
                                  key='pillbox_images/3bdd37a9-ecb9-36f2-e054-00144ff88e88.jpg')

The reason you have not seen any errors with creating the first_object variable is that Boto3 doesn’t make calls to AWS to create the reference. 

The bucket_name and the key are called identifiers, and they are the necessary parameters to create an Object. Any other attribute of an Object, such as its size, is lazily loaded. 

This means that for Boto3 to get the requested attributes, it has to make calls to AWS.

## Uploading a File

There are three ways you can upload a file:

- From an Object instance
- From a Bucket instance
- From the client

In each case, you have to provide the Filename, which is the path of the file you want to upload.

#### Object Instance Version

In [10]:
file_object.upload_file('pillbox_images/3bdd37a9-ecb9-36f2-e054-00144ff88e88.jpg')

## Downloading a File

To download a file from S3 locally, you’ll follow similar steps as you did when uploading. But in this case, the Filename parameter will map to your desired local path. 

This time, it will download the file to the tmp directory:

In [17]:
s3_resource.Object(second_bucket_name, 'pillbox_images/3bdd37a9-ecb9-36f2-e054-00144ff88e88.jpg').download_file('./test_s3file.jpg')